In [ ]:
#importing really useful stuff
import rebound
import numpy as np
import matplotlib as plt
from math import log10, floor

Section 1.0: finding and removing all unstable systems

Unstable systems will result in significantly increased transit timing variations, which could skew the averaged data. 

The critical a value (semi-major axis) for "venus" was determined to the value when the aphelion of "venus"'s orbit woukd cross with the perihelion of "earth"'s orbit for each value of 'e' (eccentricity). Equation shown below:

a_venus(1 + e_venus) = a_earth(1 - e_earth) --> a_venus = 1(1 - 0.05)/1 + e_venus


This ruled out a number of cases. However. there were still many unstable orbits on the boundary of that line (imagine a 2-dimensional graph with 'e' represented on the x axis and 'a' on the y axis', with the line represented at a=0.95/1+e). 

Due to this we ran trial cases (running the orbit systems for 100,000 years with the minimum exit distance set as 0.05 AU) for e = 0.05, e=0.1, e=0.15 and e=0.2. All cases for 'a' were run starting with a = 0.85 and running until there were 7-16 stable cases in a row. 'f' (anomaly) and 'omega' (location of particle) were randomized for each case between 0.0 and 2pi, which is why many stable orbits had to be seen in a row. 

The lowest percent of the critical a value which was a boundary for stable vs unstable cases was found to be ~0.94 (in other words, all cases lower than 0.94 x 'a crit' for each 'e' value were stable). The cutoff for cases was set to 0.93 x 'a crit'. 

All cases that remained with transit variation (in hours) greater than 9 hours were retested (running the orbit system for 100,000 years, minimum exit distance also 0.05 AU) and all were shown to be stable without exception.

In [ ]:
def runsimulation():
    
    #setup simulation
    sim = rebound.Simulation()
    sim.integrator = "ias15"
    sim.exit_min_distance = 0.05 #if the two particles ("earth" and "venus") reach within 0.05AU of each other, a rebound.encounter error is shown
    sim.add(m=1)
    sim.add(m=m_mass, a=a_distance, e=e_eccentricity, f=f_anomaly, omega=omega_locationofparticle)
    sim.add(m=3e-6, a=1,e=0.05,omega=0.25)
    sim.move_to_com()
     
    
    stable = True
    if a_distance*(1+e_eccentricity) < 1*(1-0.05): #a(1-e) for earth. if the aphelion of venus is less than the perihelion of earth, otherwise assume they're going to be unstable as the orbits cross
        print("running")
        nyears = 100000 #number of years the program runs for
        noutput = 10 #number of total steps, doesn't matter because the program checks every timestep anyways, this just sets an amount of time to integrate for
        times = np.linspace(0,nyears*2*np.pi,noutput)
        min_distance = 5
        distances = np.zeros(noutput)
        try:
            for i,time in enumerate(times):
                sim.integrate(time)
        except rebound.Encounter as error:
            print(a_distance, f_anomaly, omega_locationofparticle, "UNSTABLE") #important to print f and omega because they are randomized
            stable = False
    else: #if the orbits cross
        stable = False
        print(a_distance, "unstable. case omitted: orbits cross.")
        
    if stable == True:
        print(a_distance, f_anomaly, omega_locationofparticle, "STABLE")


#main
import numpy as np
import random
m_mass = 2.45e-6
e_eccentricity = 0.05 #0.10, 0.15, 0.20 
for i in range(85):
    a_distance = 0.85 - i*0.0015
    f_anomaly = random.uniform(0, 2*np.pi) #chooses a random value between 0 and 2pi
    omega_locationofparticle = random.uniform(0, 2*np.pi)
    runsimulation()

Section 1.1: running all systems and finding transit timing variations

With 'a' running from 0.7-0.85 AU, 'e' from 0.0-0.2, and 'f' and 'omega' from 0.0-2pi, each assumed stable system was run for 19 years (4 years kepler, 11 years pause, 4 years mission). These 19 variations along with the 'a' and 'e' of each case were recorded in a file (ttvdata2.txt). 

In [ ]:
ttvdata = open("ttvdata.txt", "w+")  #currently the data is stored in ttvdata2.txt, name of file set as ttvdata.txt in order to not accidentally delete data in ttvdata2.txt
ttvdata.close()

def ttv():
    #setup simulation
    sim = rebound.Simulation()
    sim.add(m=1) #this is the sun
    sim.add(m=m_mass, a=a_distance, e=e_eccentricity, f=f_anomaly, omega=omega_locationofparticle) #"venus", whatever parameters are currently running
    sim.add(m=3e-6, a=1,e=0.05,omega=0.25)
    sim.move_to_com()
    p = sim.particles
    
    addtottv = True
    
    #code used to find transit time below copied from hannorein
    #find individual time of intersection N times
    N=19
    transittimes = np.zeros(N)
    i = 0
    while i<N:
        y_old = p[2].y - p[0].y  
        t_old = sim.t
        sim.integrate(sim.t+0.5) # check for transits every 0.5 time units. Note that 0.5 is shorter than one orbit
        t_new = sim.t
        if y_old*(p[2].y-p[0].y)<0. and p[2].x-p[0].x>0.:   # sign changed (y_old*y<0), planet in front of star (x>0)
            while t_new-t_old>1e-7:   # bisect until prec of 1e-5 reached
                if y_old*(p[2].y-p[0].y)<0.:
                    t_new = sim.t
                else:
                    t_old = sim.t
                sim.integrate( (t_new+t_old)/2.)
            transittimes[i] = sim.t
            i += 1
            sim.integrate(sim.t+0.05)  

    
    #remove linear trend
    A = np.vstack([np.ones(N), range(N)]).T
    c, m = np.linalg.lstsq(A, transittimes)[0]  
    
    
    if a_distance > 0.93*(0.95/1+e_eccentricity):
        addtottv = False

    def round_sig(x, sig=2):
        return round(x, sig-int(floor(log10(abs(x))))-1)

    #if orbits are stable, adds the transit variations to a .txt file
    if addtottv == True:
        #store values in variable called temp. then convert to tempvconvert (each variation on indv. line, 4 sig. figs.)
        temp = (transittimes-m*np.array(range(N))-c)*(24.*365./2./np.pi);
        tempconvert = "\n" + str(a_distance) + "\n" + str(e_eccentricity)
        for i in range(len(temp)):
            tempconvert = tempconvert + "\n" + str(round_sig(temp[i], 4))
    
        #add tempconvert data to a .txt file
        ttvdata = open("ttvdata.txt","a+") 
        ttvdata.write(tempconvert)
        ttvdata.close()
       

#main (running parameters)
a_distance_min = 0.7
a_distance_max = 0.85
a_distance_total_steps = 100
a_distance_step = (a_distance_max - a_distance_min)/a_distance_total_steps

e_eccentricity_min = 0.0
e_eccentricity_max = 0.2
e_eccentricity_total_steps = 5 
e_eccentricity_step = (e_eccentricity_max - e_eccentricity_min)/e_eccentricity_total_steps

f_anomaly_min = 0.0
f_anomaly_max = 2*np.pi
f_anomaly_total_steps = 10
f_anomaly_step = (f_anomaly_max - f_anomaly_min)/f_anomaly_total_steps

omega_locationofparticle_min = 0.0
omega_locationofparticle_max = 2*np.pi
omega_locationofparticle_total_steps = 10
omega_locationofparticle_step = (omega_locationofparticle_max - omega_locationofparticle_min)/omega_locationofparticle_total_steps

m_mass = 2.45e-6
for i in range(a_distance_total_steps):
    a_distance = a_distance_min + i*a_distance_step 
    for j in range(e_eccentricity_total_steps):
        e_eccentricity = e_eccentricity_min + j*e_eccentricity_step
        for k in range(f_anomaly_total_steps): 
            f_anomaly = f_anomaly_min + k*f_anomaly_step
            for l in range(omega_locationofparticle_total_steps):
                omega_locationofparticle = omega_locationofparticle_min + l*omega_locationofparticle_step
                ttv()